In [ ]:
# ---
# title: 10. Black-Litterman Model
# tags: [Optimization, BlackLitterman, Bayesian, Finance]
# difficulty: Advanced
# ---

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Import our optimizer
import sys
sys.path.append('..')
from src.analytics.advanced_optimizer import AdvancedOptimizer
from src.analytics.portfolio_optimizer import PortfolioOptimizer

sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Black-Litterman Portfolio Optimization

## Introduction

The **Black-Litterman Model** is a sophisticated portfolio optimization framework that combines:
- **Market Equilibrium** (CAPM-based prior)
- **Investor Views** (subjective beliefs about asset returns)
- **Bayesian Statistics** (to blend prior and views into posterior estimates)

### Why Black-Litterman?

Traditional Mean-Variance Optimization (MVO) has critical flaws:
1. **Garbage In, Garbage Out**: Requires precise expected return estimates (which are notoriously unreliable)
2. **Extreme Portfolios**: Small errors in inputs lead to extreme, concentrated positions
3. **High Turnover**: Unstable weights cause excessive trading costs

**Black-Litterman solves this by**:
- Starting with a neutral **market equilibrium** (CAPM implied returns)
- Allowing investors to express **views** on specific assets
- Blending views with equilibrium using **Bayesian inference**
- Producing **stable, intuitive portfolios** that tilt toward views while respecting market wisdom

## The Black-Litterman Framework

### Step 1: Market Equilibrium (Prior)

The **implied equilibrium returns** are calculated using reverse optimization:

$$\Pi = \lambda \Sigma w_{mkt}$$

Where:
- $\Pi$ = Implied excess returns (prior)
- $\lambda$ = Risk aversion coefficient (typically 2.5)
- $\Sigma$ = Covariance matrix of returns
- $w_{mkt}$ = Market capitalization weights

### Step 2: Investor Views (Likelihood)

Views are expressed as:

$$P \cdot \mu = Q + \epsilon, \quad \epsilon \sim N(0, \Omega)$$

Where:
- $P$ = "Pick matrix" (which assets the view applies to)
- $Q$ = Expected returns from views
- $\Omega$ = Uncertainty in views (diagonal matrix)

### Step 3: Posterior Distribution (Blended Estimate)

The **posterior expected returns** combine prior and views:

$$E[R] = \left[(\tau\Sigma)^{-1} + P^T\Omega^{-1}P\right]^{-1} \left[(\tau\Sigma)^{-1}\Pi + P^T\Omega^{-1}Q\right]$$

Where $\tau$ is a scalar (typically 0.025) representing uncertainty in the prior.

### Step 4: Optimization

Use posterior returns $E[R]$ in standard mean-variance optimization to get optimal weights.

## Step 1: Load Market Data

In [ ]:
# Load returns from silver layer
silver_path = Path("../data/silver")
files = list(silver_path.glob("market_returns_*.parquet"))

if not files:
    raise FileNotFoundError("No returns data found. Run data ingestion first.")

latest = max(files, key=lambda f: f.stat().st_mtime)
returns = pd.read_parquet(latest)

print(f"Loaded returns for {len(returns.columns)} tickers")
print(f"Date range: {returns.index[0]} to {returns.index[-1]}")

# Use subset for demonstration
returns_subset = returns.iloc[:, :15]  # First 15 tickers
tickers = returns_subset.columns.tolist()
print(f"\nUsing {len(tickers)} tickers: {', '.join(tickers[:5])}...")

## Step 2: Market Equilibrium (No Views)

First, let's see what happens with **no investor views** - we get the market equilibrium portfolio.

In [ ]:
# Initialize optimizer
optimizer = AdvancedOptimizer()

# Create mock market caps (in practice, use real market cap data)
# For demonstration, we'll use random weights that sum to 1
np.random.seed(42)
market_caps = pd.Series(np.random.dirichlet(np.ones(len(tickers))), index=tickers)
market_caps = market_caps / market_caps.sum()  # Normalize

print("=== Market Capitalization Weights ===")
print(market_caps.sort_values(ascending=False))

# Black-Litterman with NO views (returns equilibrium)
bl_weights_no_views = optimizer.get_black_litterman_weights(
    returns_subset, 
    market_caps=market_caps,
    view_dict=None  # No views
)

print("\n=== Black-Litterman Weights (No Views) ===")
print(bl_weights_no_views.sort_values(ascending=False))
print(f"\n✅ With no views, BL returns market equilibrium weights")

## Step 3: Adding Investor Views

Now let's express some **subjective views** about expected returns.

### Example Views:
1. **Bullish on Tech**: We believe the first ticker will outperform by 5%
2. **Bearish on Energy**: We believe another ticker will underperform by -3%
3. **Neutral on Others**: No specific views

In [ ]:
# Define views (ticker: expected excess return)
# These are EXCESS returns (above equilibrium)
views = {
    tickers[0]: 0.05,   # Bullish: +5% excess return
    tickers[2]: -0.03,  # Bearish: -3% excess return
    tickers[5]: 0.02    # Mildly bullish: +2% excess return
}

# Confidence levels (0 to 1, where 1 = very confident)
confidences = [0.8, 0.6, 0.5]  # High, Medium, Low confidence

print("=== Investor Views ===")
for i, (ticker, ret) in enumerate(views.items()):
    conf = confidences[i]
    sentiment = "Bullish" if ret > 0 else "Bearish"
    print(f"{ticker}: {sentiment} ({ret:+.1%}) - Confidence: {conf:.0%}")

# Apply Black-Litterman with views
bl_weights_with_views = optimizer.get_black_litterman_weights(
    returns_subset,
    market_caps=market_caps,
    view_dict=views,
    confidences=confidences,
    risk_aversion=2.5
)

print("\n=== Black-Litterman Weights (With Views) ===")
print(bl_weights_with_views.sort_values(ascending=False))

## Step 4: Visualize Weight Changes

Let's see how our views shifted the portfolio from equilibrium.

In [ ]:
# Create comparison dataframe
weight_comparison = pd.DataFrame({
    'Market Equilibrium': market_caps,
    'BL (No Views)': bl_weights_no_views,
    'BL (With Views)': bl_weights_with_views
}).fillna(0)

# Calculate weight changes
weight_comparison['Change from Equilibrium'] = (
    weight_comparison['BL (With Views)'] - weight_comparison['Market Equilibrium']
)

# Plot comparison
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Subplot 1: Weight comparison
weight_comparison[['Market Equilibrium', 'BL (With Views)']].plot(
    kind='bar', ax=axes[0], color=['steelblue', 'coral'], alpha=0.8
)
axes[0].set_title('Portfolio Weights: Equilibrium vs Black-Litterman', 
                  fontsize=14, fontweight='bold')
axes[0].set_ylabel('Weight', fontsize=12)
axes[0].set_xlabel('')
axes[0].legend(loc='upper right')
axes[0].grid(axis='y', alpha=0.3)
axes[0].axhline(y=0, color='black', linewidth=0.8)

# Subplot 2: Weight changes
weight_comparison['Change from Equilibrium'].plot(
    kind='bar', ax=axes[1], color='green', alpha=0.7
)
axes[1].set_title('Weight Changes Due to Views', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Weight Change', fontsize=12)
axes[1].set_xlabel('Ticker', fontsize=12)
axes[1].grid(axis='y', alpha=0.3)
axes[1].axhline(y=0, color='black', linewidth=0.8)

# Highlight tickers with views
for ticker in views.keys():
    idx = tickers.index(ticker)
    axes[1].get_children()[idx].set_color('darkgreen')
    axes[1].get_children()[idx].set_alpha(1.0)

plt.tight_layout()
plt.show()

print("\n📊 Interpretation:")
print("- Green bars show how views tilted the portfolio")
print("- Darker green = tickers with explicit views")
print("- Other tickers adjust to maintain diversification")

## Step 5: Sensitivity to Confidence Levels

Let's explore how **confidence** in our views affects the portfolio.

In [ ]:
# Test different confidence levels
confidence_scenarios = {
    'Low Confidence': [0.3, 0.3, 0.3],
    'Medium Confidence': [0.6, 0.6, 0.6],
    'High Confidence': [0.9, 0.9, 0.9]
}

results = {'Market Equilibrium': market_caps}

for scenario_name, conf_levels in confidence_scenarios.items():
    weights = optimizer.get_black_litterman_weights(
        returns_subset,
        market_caps=market_caps,
        view_dict=views,
        confidences=conf_levels
    )
    results[scenario_name] = weights

# Create comparison dataframe
confidence_df = pd.DataFrame(results).fillna(0)

# Plot for tickers with views
view_tickers = list(views.keys())
confidence_df.loc[view_tickers].T.plot(
    kind='bar', figsize=(12, 6), alpha=0.8
)
plt.title('Impact of Confidence Levels on Portfolio Weights', 
          fontsize=14, fontweight='bold')
plt.ylabel('Weight', fontsize=12)
plt.xlabel('Scenario', fontsize=12)
plt.legend(title='Ticker', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='y', alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\n📊 Key Insight:")
print("- Higher confidence → Portfolio tilts MORE toward views")
print("- Lower confidence → Portfolio stays CLOSER to equilibrium")
print("- This prevents overconfident views from dominating the portfolio")

## Step 6: Compare BL vs MVO

Let's compare Black-Litterman with traditional Mean-Variance Optimization.

In [ ]:
# Compute MVO weights
mvo_optimizer = PortfolioOptimizer()
mvo_weights = mvo_optimizer.get_max_sharpe_weights(returns_subset)

# Compare all strategies
all_strategies = pd.DataFrame({
    'Market Equilibrium': market_caps,
    'Black-Litterman': bl_weights_with_views,
    'MVO (Max Sharpe)': mvo_weights
}).fillna(0)

# Plot comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for i, (col, color) in enumerate(zip(all_strategies.columns, 
                                      ['steelblue', 'coral', 'green'])):
    all_strategies[col].sort_values(ascending=True).plot(
        kind='barh', ax=axes[i], color=color, alpha=0.8
    )
    axes[i].set_title(col, fontsize=12, fontweight='bold')
    axes[i].set_xlabel('Weight', fontsize=10)
    axes[i].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate concentration
def herfindahl_index(weights):
    return (weights ** 2).sum()

print("\n=== Portfolio Concentration (Herfindahl Index) ===")
for col in all_strategies.columns:
    hhi = herfindahl_index(all_strategies[col])
    print(f"{col:20s}: {hhi:.4f}")
print("\n(Lower = More diversified)")

## Step 7: Performance Metrics

In [ ]:
def portfolio_stats(weights, returns, name="Portfolio"):
    """Calculate portfolio statistics"""
    portfolio_returns = (returns * weights).sum(axis=1)
    
    ann_return = portfolio_returns.mean() * 252
    ann_vol = portfolio_returns.std() * np.sqrt(252)
    sharpe = ann_return / ann_vol if ann_vol > 0 else 0
    
    return {
        'Strategy': name,
        'Annual Return': ann_return,
        'Annual Volatility': ann_vol,
        'Sharpe Ratio': sharpe
    }

# Calculate stats
stats = []
for col in all_strategies.columns:
    stats.append(portfolio_stats(all_strategies[col], returns_subset, col))

stats_df = pd.DataFrame(stats).set_index('Strategy')

print("\n=== Portfolio Performance Comparison ===")
print(stats_df.to_string())

# Visualize
stats_df.plot(kind='bar', figsize=(12, 6), rot=0)
plt.title('Portfolio Performance Metrics', fontsize=14, fontweight='bold')
plt.ylabel('Value', fontsize=12)
plt.legend(loc='upper left')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## Key Takeaways

### ✅ Advantages of Black-Litterman:
1. **Intuitive Framework**: Start with market equilibrium, add views incrementally
2. **Stability**: Portfolios are stable and don't change drastically with small input changes
3. **Flexibility**: Can express partial views (not all assets need views)
4. **Confidence Weighting**: Incorporates uncertainty in views naturally
5. **Diversification**: Maintains diversification while tilting toward views

### ⚠️ Limitations:
1. **Complexity**: More complex than MVO (requires understanding of Bayesian inference)
2. **Parameter Sensitivity**: Results depend on $\tau$, $\lambda$, and $\Omega$ choices
3. **View Specification**: Requires careful thought about how to express views
4. **Market Cap Dependency**: Needs market cap data (or proxy) for equilibrium

### 🎯 When to Use Black-Litterman:
- When you have **subjective views** but want to respect market wisdom
- For **institutional portfolios** where stability and explainability matter
- When **expected returns are uncertain** (which is almost always)
- In **tactical asset allocation** with changing views over time

### 🔬 Practical Tips:
1. **Start Conservative**: Use lower confidence levels initially
2. **Limit Views**: Don't express views on every asset (defeats the purpose)
3. **Backtest Views**: Validate that your views actually add value historically
4. **Monitor Drift**: Rebalance when portfolio drifts from target weights

### 📚 Further Reading:
- Black, F. & Litterman, R. (1992). "Global Portfolio Optimization"
- Idzorek, T. (2005). "A Step-by-Step Guide to the Black-Litterman Model"
- Walters, J. (2014). "The Black-Litterman Model in Detail"